In [1]:
import torch.nn as nn
import gradio as gr
import requests
import pandas as pd
import torch
from transformers import BertTokenizer
from transformers import BertModel


class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes, dropout_prob=0.1):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(dropout_prob)
        
        self.fc = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),  
            nn.ReLU(),  
            nn.LayerNorm(512), 
            nn.Dropout(dropout_prob),
            nn.Linear(512, num_classes)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_states = outputs.last_hidden_state
        pooled_output = last_hidden_states.mean(dim=1) 
        
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        
        return logits


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bert_model_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, 2).to(device)
model.load_state_dict(torch.load("model/bert_classifier.pth", map_location=device))
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [3]:
API_KEY = '550200ce3eae4e808f1924e695de35ad'
API_URL = 'https://newsapi.org/v2/everything'

def extract_data(query):
    params = {
        'q': query,
        'apiKey': API_KEY,
    }
    response = requests.get(API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        articles_df = pd.DataFrame(data.get('articles', []))
        return articles_df
    else:
        print("Error: Unable to fetch data. Status code:", response.status_code)
        return pd.DataFrame()
    
def predict_verdict(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        return "TRUE" if preds.item() == 1 else "FAKE"


def interface(query):
    prediction = predict_verdict(query, model, tokenizer, device) 
    return prediction 


iface = gr.Interface(
    fn=interface,
    inputs=[
        gr.Textbox(label="Enter News Text"),
    ],
    outputs="text")

iface.launch(server_port=7862, server_name="0.0.0.0", share=True)


Running on local URL:  http://0.0.0.0:7862
Running on public URL: https://b6149d6b20a9fc1eda.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
